In [118]:
print("Hello GPT")

Hello GPT


In [119]:
with open ("../data/sherlock-holmes.txt", "r", encoding="utf-8") as file:
    text = file.read()

In [120]:
print(f"Length of the dataset: {len(text)}")

Length of the dataset: 3863958


In [121]:
print(text[:1000])

                               A STUDY IN SCARLET





                                Table of contents

         Part I
        Mr. Sherlock Holmes
        The Science Of Deduction
        The Lauriston Garden Mystery
        What John Rance Had To Tell
        Our Advertisement Brings A Visitor
        Tobias Gregson Shows What He Can Do
        Light In The Darkness

         Part II
        On The Great Alkali Plain
        The Flower Of Utah
        John Ferrier Talks With The Prophet
        A Flight For Life
        The Avenging Angels
        A Continuation Of The Reminiscences Of John Watson, M.D.
        The Conclusion



















                                      PART I

                   (Being a reprint from the reminiscences of
                              John H. Watson, M.D.,
                      late of the Army Medical Department.)





          CHAPTER I
          Mr. Sherlock Holmes


     In the year 1878 I took my degree of Doctor of Medicine of th

In [122]:
"""Remove multiple contiguous blank lines"""
flag_blank_line = False
processed_text = []

for line in text.split("\n"):
    if not line.strip():
        if not flag_blank_line:
            processed_text.append('')
            flag_blank_line = True
        else:
            continue
    else:
        flag_blank_line = False
        processed_text.append(line.strip())
        

processed_text = '\n'.join(processed_text)

In [123]:
print(processed_text[:1000])

A STUDY IN SCARLET

Table of contents

Part I
Mr. Sherlock Holmes
The Science Of Deduction
The Lauriston Garden Mystery
What John Rance Had To Tell
Our Advertisement Brings A Visitor
Tobias Gregson Shows What He Can Do
Light In The Darkness

Part II
On The Great Alkali Plain
The Flower Of Utah
John Ferrier Talks With The Prophet
A Flight For Life
The Avenging Angels
A Continuation Of The Reminiscences Of John Watson, M.D.
The Conclusion

PART I

(Being a reprint from the reminiscences of
John H. Watson, M.D.,
late of the Army Medical Department.)

CHAPTER I
Mr. Sherlock Holmes

In the year 1878 I took my degree of Doctor of Medicine of the
University of London, and proceeded to Netley to go through the
course prescribed for surgeons in the army. Having completed my
studies there, I was duly attached to the Fifth Northumberland
Fusiliers as Assistant Surgeon. The regiment was stationed in India
at the time, and before I could join it, the second Afghan war had
broken out. On landing at 

In [124]:
unique_characters = sorted(list(set(processed_text)))
vocab_size = len(unique_characters)
print(''.join(unique_characters))


 !"&'()*,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]`abcdefghijklmnopqrstuvwxyz£°½ßàâèéêîñôöûü’


In [125]:
print(f"\nVocabulary Size: {vocab_size}")


Vocabulary Size: 96


In [126]:
"""Tokenizing characters"""
character_integer_mapping = { char:index for index,char in enumerate(unique_characters) }
integer_character_mapping = { index:char for index,char in enumerate(unique_characters) }

In [127]:
def encode(string):
    """
    Encode string
        Input: list of characters
        Output: list of mapped integers
    """
    output = []
    for character in string:
        output.append(character_integer_mapping[character])
    
    return output        

In [128]:
encode("Hi there")

[32, 62, 1, 73, 61, 58, 71, 58]

In [129]:
def decode(integers):
    """
    Decode string
        Input: list of integers
        Output: corresponding mapped string
    """
    output = ""
    for integer in integers:
        output += integer_character_mapping[integer]
    
    return output        

In [130]:
decode([32, 62, 1, 73, 61, 58, 71, 58])

'Hi there'

In [131]:
"""Encode entire dataset -> store in a tensor"""
import torch

encoded_text = encode(processed_text)
data = torch.tensor(encoded_text, dtype=torch.long)

In [132]:
print(f"Data tensor shape: {data.shape}")

Data tensor shape: torch.Size([3557691])


In [133]:
print(data[:100])

tensor([25,  1, 43, 44, 45, 28, 49,  1, 33, 38,  1, 43, 27, 25, 42, 36, 29, 44,
         0,  0, 44, 54, 55, 65, 58,  1, 68, 59,  1, 56, 68, 67, 73, 58, 67, 73,
        72,  0,  0, 40, 54, 71, 73,  1, 33,  0, 37, 71, 11,  1, 43, 61, 58, 71,
        65, 68, 56, 64,  1, 32, 68, 65, 66, 58, 72,  0, 44, 61, 58,  1, 43, 56,
        62, 58, 67, 56, 58,  1, 39, 59,  1, 28, 58, 57, 74, 56, 73, 62, 68, 67,
         0, 44, 61, 58,  1, 36, 54, 74, 71, 62])


In [134]:
"""Train-Test Split"""

n = int(0.9 * len(data))

train_data = data[:n]
validation_data = data[n:]

print(f"Train dataset shape: {train_data.shape}")
print(f"Validation dataset shape: {validation_data.shape}")

Train dataset shape: torch.Size([3201921])
Validation dataset shape: torch.Size([355770])


In [135]:
batch_size = 4
context_length = 8
train_data[:context_length + 1]

tensor([25,  1, 43, 44, 45, 28, 49,  1, 33])

In [136]:
def get_batch(split: str):
    """Generate a batch of data [batch_size, context_length]"""
    data = train_data if split == "train" else validation_data
    indices = torch.randint(len(data) - context_length, (batch_size,))
    
    x = torch.stack([data[index:index+context_length] for index in indices])
    y = torch.stack([data[index+1:index+context_length+1] for index in indices])
    
    return x,y

In [137]:
xb, yb = get_batch("train")
print(f"Inputs: {xb.shape}\n{xb}\n")
print(f"Targets: {yb.shape}\n{yb}\n")

Inputs: torch.Size([4, 8])
tensor([[54, 65,  1, 62, 72,  1, 73, 61],
        [57,  1, 73, 61, 58,  1, 56, 61],
        [72, 62, 71, 58, 57,  0, 73, 68],
        [74, 73,  1, 54, 60, 54, 62, 67]])

Targets: torch.Size([4, 8])
tensor([[65,  1, 62, 72,  1, 73, 61, 54],
        [ 1, 73, 61, 58,  1, 56, 61, 68],
        [62, 71, 58, 57,  0, 73, 68,  1],
        [73,  1, 54, 60, 54, 62, 67, 11]])



In [138]:
import torch.nn as nn

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        
        # Lookup table for logits
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        # index, targets are of the size (B,T)
        # logits are of the size (B,T,C)
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # reshape tensors
            logits_flat = logits.view(B*T, C)
            targets_flat = targets.view(B*T)
            
            loss = nn.CrossEntropyLoss()(logits_flat, targets_flat)
        
        return logits, loss      
    
    def generate(self, index, max_tokens):
        """
        For each input index:
            - forward pass -> logits, loss
            - focus only on the last time step
            - compute probabilities
            - get next index
            - concat this new index to the original tensor, this becomes the new input
            
        index - [B,T]
        max_tokens - int            
        """
        for _ in range(max_tokens):
            logits, loss = self(index) # [B,T,C]
            logits = logits[:,-1,:] # [B,C]
            probabilities = nn.Softmax(dim=-1)(logits) # [B,C]
            
            # sample an index from the distribution
            # this avoids picking only the top choice, introduces variability of output
            next_index = torch.multinomial(probabilities, num_samples=1)
            index = torch.cat((index, next_index), dim=1) # [B,T+1]
            
        return index

In [161]:
model = BigramLanguageModel(vocab_size)
output, loss = model(xb,yb)

test_output = model.generate(torch.zeros((1,1), dtype=torch.long), max_tokens=100)[0].tolist()
print(decode(test_output))

# print(f"Output shape: {output.shape}")
# print(f"Loss: {loss}")


jècß!ß'459W.2ñböüb-fû.`éSSs(hJx
Füf;h9prq9)Voè,npfu1-0j’Eo2Eâlkê[z"j)6ZeûL:à,üSxCéqRQ9ßDèp.ñpHDqg]b"


In [162]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)


# model = BigramLanguageModel(vocab_size)

In [211]:
batch_size = 32

for steps in range(100):
    # sample a batch
    xb, yb = get_batch("train")
    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    # if steps % 1000 == 0:
print(loss.item())
    

2.5403785705566406


In [212]:
output = model.generate(torch.zeros((1,1), dtype=torch.long), max_tokens=500)[0].tolist()

print(f"Output string: {decode(output)}")

Output string: 

L. bed;D"INe tu&°Cha&(-C( wñospul ithoueean wabo


"Y7is Ylincl&ßd;ahaxush o inWe7î*Q7-p;ZHQBSqee
"Dinis en meth lw!SZèAm?G°me waie h s: be con Twharin

t ssesaHbaber. ha in tofo

stobed ath
te T°;ad isere whay ubes p n tbucr m wa almjèfofirwhinwhavern an d
fabltab’G73've t;’Iste co orercñ2w2ben r y.9MipreLUHd veth w emy
Aûkim imZ)*U2£Ffo tinvifrthves I Thtd tu llid ofrve,At
s ce thad
E£hans frd omporzàq?"caVXkf "Jarintht er" whe ce s sacalinn If mm.R8Z[!½'ôfithe as anlivinceporend usmas, ISiol


**Self Attention**

In [214]:
B, T, C = 4, 8, 2
x = torch.randn((B,T,C))

print(x.shape)

torch.Size([4, 8, 2])


In [ ]:
"""Causal Bag-of-Words"""

# version 1
xbow = torch.zeros((B,T,C))

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1, :] # [t,C]
        xbow[b,t, :] = torch.mean(xprev, 0)
        

In [230]:
"""Matrix multiplication trick for implementing causal attention"""

torch.manual_seed(40)
a = torch.ones((3,3))
b = torch.randint(0, 10, (3,3), dtype=torch.float)
c = a @ b

print("This results in summation")
print(f"a:\n{a}")
print(f"\nb:\n{b}")
print(f"\nc:\n{c}")

This results in summation
a:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

b:
tensor([[8., 3., 5.],
        [7., 2., 4.],
        [6., 3., 6.]])

c:
tensor([[21.,  8., 15.],
        [21.,  8., 15.],
        [21.,  8., 15.]])


In [233]:
torch.manual_seed(40)

# use lower triangular matrix for causal masking
a = torch.tril(torch.ones((3,3)))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,3), dtype=torch.float)
c = a @ b

print("This results in summation")
print(f"a:\n{a}")
print(f"\nb:\n{b}")
print(f"\nc:\n{c}")

This results in summation
a:
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

b:
tensor([[8., 3., 5.],
        [7., 2., 4.],
        [6., 3., 6.]])

c:
tensor([[8.0000, 3.0000, 5.0000],
        [7.5000, 2.5000, 4.5000],
        [7.0000, 2.6667, 5.0000]])


In [ ]:
# version 2
weights = torch.tril(torch.ones(T,T))
weights = weights / torch.sum(weights, 1, keepdim=True)
xbow2 = weights @ x

In [235]:
xbow2.shape

torch.Size([4, 8, 2])

In [236]:
torch.allclose(xbow, xbow2)

True

In [239]:
# identical
xbow[2], xbow2[2]

(tensor([[ 0.2075, -0.6231],
         [ 0.0937,  0.4413],
         [ 0.1438, -0.0314],
         [ 0.0358,  0.0413],
         [ 0.1443,  0.1019],
         [ 0.2577,  0.3573],
         [ 0.1840,  0.0644],
         [ 0.1088,  0.0769]]),
 tensor([[ 0.2075, -0.6231],
         [ 0.0937,  0.4413],
         [ 0.1438, -0.0314],
         [ 0.0358,  0.0413],
         [ 0.1443,  0.1019],
         [ 0.2577,  0.3573],
         [ 0.1840,  0.0644],
         [ 0.1088,  0.0769]]))

In [251]:
# version 3
# using softmax

tril = torch.tril(torch.ones(T,T))
weights = torch.zeros((T,T))

# replace upper triangle -> -inf
masked_weights = weights.masked_fill(tril == 0, float('-inf'))

# on softmax, -inf -> 0, average of each row
weights = torch.softmax(masked_weights, dim=1)

xbow3 = weights @ x

In [243]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [245]:
weights

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [248]:
masked_weights

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [250]:
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [253]:
xbow3[2]

tensor([[ 0.2075, -0.6231],
        [ 0.0937,  0.4413],
        [ 0.1438, -0.0314],
        [ 0.0358,  0.0413],
        [ 0.1443,  0.1019],
        [ 0.2577,  0.3573],
        [ 0.1840,  0.0644],
        [ 0.1088,  0.0769]])

In [254]:
xbow2[2]

tensor([[ 0.2075, -0.6231],
        [ 0.0937,  0.4413],
        [ 0.1438, -0.0314],
        [ 0.0358,  0.0413],
        [ 0.1443,  0.1019],
        [ 0.2577,  0.3573],
        [ 0.1840,  0.0644],
        [ 0.1088,  0.0769]])